# Проверка написан ли текст ИИ

In [ ]:
import 'dotenv';
import { GigaChat } from 'gigachat';
import * as dotenv from 'dotenv';
import { Agent } from 'node:https';

const httpsAgent = new Agent({
  rejectUnauthorized: false,
});

dotenv.config();
const client = new GigaChat({
  timeout: 600,
  model: 'GigaChat',
  httpsAgent: httpsAgent,
});

In [39]:
const text = `Котики — это милые и пушистые домашние животные, которые относятся к семейству кошачьих. 
Они известны своим дружелюбным характером, игривостью и мягкой шерстью. 
Котики могут быть разных пород, каждая из которых имеет свои уникальные особенности. 
А этот текст писала точно не нейросеть, 
потому что это пишу я. У меня особо нечего сказать про котов,
кроме того, что они мягкие и пушистые, мурчащие существа.
Эти животные стали популярными благодаря своей способности поднимать настроение людям 
и создавать уютную атмосферу в доме.`
const aiResult = await client.aiCheck(text, "GigaCheckDetection")
aiResult

{
  category: "mixed",
  characters: 533,
  tokens: 224,
  ai_intervals: [ [ 1, 262 ], [ 415, 533 ] ],
  xHeaders: {
    xRequestID: "1ba41150-e8a5-4128-a884-d6c38c8d500e",
    xSessionID: "3e5537d9-f80e-4810-8015-e5bc6bc1c164",
    xClientID: undefined
  }
}

In [40]:
function aiIntervalsToArgs(text, aiIntervals) {
    if (!aiIntervals.length) return [text];
    let output = "";
    const colors = [];
    if (aiIntervals[0][0] > 0) {
        output += "%c" + text.substr(0, aiIntervals[0][0]);
        colors.push("color: black");
    }
    for (let i = 0; i < aiIntervals.length; i++) {
        const interval = aiIntervals[i];
        if (i > 0 && aiIntervals[i][0] - aiIntervals[i-1][1] > 1) {
            output += "%c" + text.substr(aiIntervals[i-1][1], aiIntervals[i][0] - aiIntervals[i - 1][1]);
            colors.push("color: black");
        }
        output += "%c" + text.substr(aiIntervals[i][0], aiIntervals[i][1] - aiIntervals[i][0]);
        colors.push("color: red");
    }
    if (aiIntervals[aiIntervals.length - 1][1] < text.length) {
         output += "%c" + text.substr(aiIntervals[aiIntervals.length - 1][1], text.length);
         colors.push("color: black");
    }
    return [output, ...colors]
}

In [41]:
console.log(...aiIntervalsToArgs(text, aiResult.ai_intervals))

Котики — это милые и пушистые домашние животные, которые относятся к семейству кошачьих. 
Они известны своим дружелюбным характером, игривостью и мягкой шерстью. 
Котики могут быть разных пород, каждая из которых имеет свои уникальные особенности. 
А этот текст писала точно не нейросеть, 
потому что это пишу я. У меня особо нечего сказать про котов,
кроме того, что они мягкие и пушистые, мурчащие существа.
Эти животные стали популярными благодаря своей способности поднимать настроение людям 
и создавать уютную атмосферу в доме.
